In [ ]:
using Revise
using Burgers
using DataDeps, MAT, MLUtils
using NeuralOperators, Flux
using BSON
using DataDeps, MAT, MLUtils
using NeuralOperators, Flux
using CUDA, FluxTraining, BSON
import Flux: params
using BSON: @save, @load
using ProgressBars
using Zygote
using Optimisers, ParameterSchedulers
using LazySets
using Burgers
using FluxTraining
using Plots

In [ ]:
function my_get_data(file_path; n = 10000, Δsamples = 1, grid_size = div(201, Δsamples), T = Float32)
    file = matopen(file_path)
    x_data = T.(collect(read(file, "a")[1:n, 1:Δsamples:end]'))
    y_data = T.(collect(read(file, "u")[1:n, 1:Δsamples:end]'))
    safe_labels = T.(collect(read(file, "safe")[1:n, 1:Δsamples:end]'))
    pf_labels = T.(collect(read(file, "pf")[1:n, 1:Δsamples:end]'))
    close(file)

    x_loc_data = Array{T, 3}(undef, 2, grid_size, n)
    x_loc_data[1, :, :] .= reshape(repeat(LinRange(0, 0.2, grid_size), n), (grid_size, n))
    x_loc_data[2, :, :] .= x_data

    return x_loc_data, reshape(y_data, 1, :, n), safe_labels, pf_labels
end

function my_get_dataloader(; ratio::Float64 = 1.0, batchsize = 1)
    𝐱1, 𝐲1, safe1, pf1 = my_get_data("data_opt_ns_abs_0.145.mat") 
    
    data_train1, data_test1 = splitobs((𝐱1, 𝐲1, safe1, pf1), at = ratio)
    𝐱2, 𝐲2, safe2, pf2 = my_get_data("data_ppo_ns_abs_0.145.mat")
    
    data_train2, data_test2 = splitobs((𝐱2, 𝐲2, safe2, pf2), at = ratio)
    𝐱3, 𝐲3, safe3, pf3 = my_get_data("data_sac_ns_abs_0.145.mat")
    
    data_train3, data_test3 = splitobs((𝐱3, 𝐲3, safe3, pf3), at = ratio)

    data_train1_x_pf = data_train1[1][:,:,:]
    data_test1_x_pf = data_test1[1][:,:,:]
    data_train1_y_pf = data_train1[2][:,:,:]
    data_test1_y_pf = data_test1[2][:,:,:]
    data_train1_safe_pf = data_train1[3][:,:]
    data_test1_safe_pf = data_test1[3][:,:]

    data_train2_x_pf = data_train2[1][:,:,:]
    data_test2_x_pf = data_test2[1][:,:,:]
    data_train2_y_pf = data_train2[2][:,:,:]
    data_test2_y_pf = data_test2[2][:,:,:]
    data_train2_safe_pf = data_train2[3][:,:]
    data_test2_safe_pf = data_test2[3][:,:]

    data_train3_x_pf = data_train3[1][:,:,:]
    data_test3_x_pf = data_test3[1][:,:,:]
    data_train3_y_pf = data_train3[2][:,:,:]
    data_test3_y_pf = data_test3[2][:,:,:]
    data_train3_safe_pf = data_train3[3][:,:]
    data_test3_safe_pf = data_test3[3][:,:]
    

    @show size(data_test1_x_pf), size(data_test2_x_pf), size(data_test3_x_pf)
    @show size(data_train1_x_pf), size(data_train2_x_pf), size(data_train3_x_pf)



    data_train = (cat(cat(data_train1_x_pf, data_train2_x_pf, dims=3), data_train3_x_pf, dims=3), 
                    cat(cat(data_train1_y_pf, data_train2_y_pf, dims=3), data_train3_y_pf, dims=3), 
                    cat(cat(data_train1_safe_pf, data_train2_safe_pf, dims=2), data_train3_safe_pf, dims=2)) # omit the last pf tumple
    data_test = (cat(cat(data_test1_x_pf, data_test2_x_pf, dims=3), data_test3_x_pf, dims=3), 
                cat(cat(data_test1_y_pf, data_test2_y_pf, dims=3), data_test3_y_pf, dims=3), 
                cat(cat(data_test1_safe_pf, data_test2_safe_pf, dims=2), data_test3_safe_pf, dims=2)) # # omit the last pf tumple
    loader_train = DataLoader(data_train, batchsize = batchsize, shuffle = false)
    loader_test = DataLoader(data_test, batchsize = batchsize, shuffle = false)

    return loader_train, loader_test
end

In [ ]:
function find_derivative_1step(vector)
    M, N = size(vector)[2], size(vector)[3]

    # Assume `vector` is the (2, M, N) array
    inputs = vector[1, :, :]  # Shape (M, N)
    outputs = vector[2, :, :]  # Shape (M, N)

    # Preallocate the derivative array with shape (1, M, N)
    derivatives = zeros(Float64, 1, M, N)

    # 1-step forward finite difference for all points from 1 to M-1
    derivatives[1, 1:M-1, :] = (outputs[2:M, :] .- outputs[1:M-1, :]) ./ (inputs[2:M, :] .- inputs[1:M-1, :])

    # 1-step backward finite difference for the last point
    derivatives[1, M, :] = (outputs[M, :] .- outputs[M-1, :]) ./ (inputs[M, :] .- inputs[M-1, :])

    # `derivatives` now contains the derivative of the output with respect to the input
    # with shape (1, M, N)
    return derivatives
end

function get_model(name)
    model_path = joinpath(@__DIR__, "./model/")
    @assert name in readdir(model_path)
    model_file = name
    return BSON.load(joinpath(model_path, model_file), @__MODULE__)
end

function reconstruct_traj(U_0, T_traj, U_dot_traj)
    # Get the dimensions of the input trajectory
    _, M, N = size(T_traj)

    # Preallocate the reconstructed trajectory array with shape (1, M, N)
    U_traj = zeros(Float64, 1, M, N)

    # Set the initial value for all trajectories
    U_traj[1, 1, :] .= U_0

    # Compute the time differences dt between consecutive time steps
    dt = T_traj[1, 2:end, :] .- T_traj[1, 1:end-1, :]

    # Calculate the cumulative sum of derivatives multiplied by dt
    # This gives the increment to add to the initial value at each step
    changes = U_dot_traj[1, 1:end-1, :] .* dt

    # Compute the cumulative sum of changes
    cumulative_changes = cumsum(changes, dims=1)

    # Reconstruct the trajectory
    U_traj[1, 2:end, :] .= U_0 .+ cumulative_changes

    # Return the reconstructed trajectory
    return U_traj
end

function reconstruct_traj_central(U_0, T_traj, U_dot_traj)
    # Get the dimensions of the input trajectory
    _, M, N = size(T_traj)

    # Preallocate the reconstructed trajectory array with shape (1, M, N)
    U_traj = zeros(Float64, 1, M, N)

    # Set the initial value for all trajectories
    U_traj[1, 1, :] .= U_0

    # Compute the time differences dt for central differences
    dt_central = T_traj[1, 3:end, :] .- T_traj[1, 1:end-2, :]  # Time differences for central points

    # Calculate the changes using central differences for the interior points (2 to M-1)
    changes_central = U_dot_traj[1, 2:M-1, :] .* dt_central

    # Forward difference for the first point
    dt_forward = T_traj[1, 2, :] .- T_traj[1, 1, :]
    changes_forward = U_dot_traj[1, 1, :] .* dt_forward

    # Backward difference for the last point
    dt_backward = T_traj[1, M, :] .- T_traj[1, M-1, :]
    changes_backward = U_dot_traj[1, M, :] .* dt_backward

    # Reconstruct the trajectory using both forward, central, and backward differences
    U_traj[1, 2, :] .= U_traj[1, 1, :] .+ changes_forward  # Forward step for the first point
    for i in eachindex(changes_central)
        U_traj[1, 2+i, :] .= U_traj[1, i, :] .+ changes_central[i,:]
    end

    U_traj[1, M, :] .= U_traj[1, M-1, :] .+ changes_backward  # Backward step for the last point

    # Return the reconstructed trajectory
    return U_traj
end


In [ ]:
using JuMP
using CPLEX

# Define a simple QP problem for CPLEX
function solve_multistep_qp_with_cplex(U_dot_nominal,G_u,G_t,phi_t,phi_Y,phiY,α,T,phiU_0;opt_alpha=false,diff_U_dot_threshold=Inf)
    n_steps = size(U_dot_nominal)[1]
    # Define the model using CPLEX
    model = Model(CPLEX.Optimizer)
    set_silent(model)
    if opt_alpha
        @variable(model, x[1:n_steps+1])  # Decision variables x1
        @objective(model, Min, (x[1:end-1]' .- U_dot_nominal') * (x[1:end-1] .- U_dot_nominal))
        @constraint(model, phi_t .+ phi_Y .* (G_t .+ G_u .* x[1:end-1]) .+ x[end] .* phiY .+ (1/T) * phiU_0 .<= 0)
        @constraint(model, x[end]  .>= 0)
    else
        C = (α * ℯ^(-α*T)) / (1-ℯ^(-α*T))
        @variable(model, x[1:n_steps])  # Decision variables x1
        @objective(model, Min, (x' .- U_dot_nominal') * (x .- U_dot_nominal))
        @constraint(model, phi_t .+ phi_Y .* (G_t .+ G_u .* x) .+ α .* phiY .+ C * phiU_0 .<= 0)
    end
    
    # Solve the QP using CPLEX
    # println("Solving with CPLEX...")
    optimize!(model)
    
    # Get the solution
    solution_cplex = value.(x)
    # println("Solution using CPLEX: ", solution_cplex)
    if abs(solution_cplex[1] - U_dot_nominal[1]) > diff_U_dot_threshold
        return U_dot_nominal[1]
    else
        return solution_cplex[1]
    end
end


Replace the pretrained neural operator path `NEURAL_OPERATOR_PATH` with the one saved in `train_ns_all_pf.jl`. Replace `CBF_PATH` with the model saved in `train_cbf_ns.ipynb`.

In [ ]:

pretrained_NO="NEURAL_OPERATOR_PATH"

train_loader, test_loader = my_get_dataloader()

model_NO = FourierNeuralOperator(ch = (2, 64, 64, 64, 64, 64, 128, 1), modes = (16,), 
                              σ = gelu)
if isnothing(pretrained_NO)
    model_NO = FourierNeuralOperator(ch = (2, 64, 64, 64, 64, 64, 128, 1), modes = (16,), 
                              σ = gelu)
else
    model_NO = get_model(pretrained_NO)[:model]
end
model_CBF = Chain(
        Dense(2 => 16, relu),   # activation function inside layer
        Dense(16 => 64, relu),   # activation function inside layer
        Dense(64 => 16, relu),   # activation function inside layer
        Dense(16 => 1)
    )



abs_flag=true

use_central_flag = true

model_CBF = get_model("CBF_PATH")[:model_CBF] 

α = 0.00001

# for ppo
max_mpc_step = 1
opt_alpha = true
diff_U_dot_threshold = 50 

# for sac
# max_mpc_step = 1
# opt_alpha = true
# diff_U_dot_threshold = 35


filter_result_list_ppo = []
ind = 1
j_index = 0
tested_num = 0
for item in train_loader
    j_index += 1
    if (j_index <= 5000)
        continue
    elseif (j_index > 5100) 
        if item[3][end, 1] == 1
            continue
        end
        if tested_num == 100
            break
        end
    end
    tested_num += 1
    @show j_index, ind
    @show ind
    x_batch = item[1]
    y_batch = item[2]
    safe_batch = item[3]
    t_0 = x_batch[1,1,1]
    U_0 = x_batch[2,1,1]
    Y_0 = y_batch[1,1,1]
    @assert U_0 == Y_0
    T = x_batch[1,end,1]
    if use_central_flag
        U_dot_nominal_traj = zeros(size(y_batch))
        U_dot_nominal_traj[1, 2:end-1, :] = (x_batch[2,3:end, :] .- x_batch[2,1:end-2, :]) ./ (x_batch[1,3:end, :] .- x_batch[1,1:end-2, :])
        U_dot_nominal_traj[1, 1, :] = (x_batch[2,2, :] .- x_batch[2,1, :]) ./ (x_batch[1,2, :] .- x_batch[1,1, :])
        U_dot_nominal_traj[1, end, :] = (x_batch[2,end, :] .- x_batch[2,end-1, :]) ./ (x_batch[1, end, :] .- x_batch[1, end-1, :])
    else
        U_dot_nominal_traj = zeros(size(y_batch))
        U_dot_nominal_traj[1, 1:end-1, :] = (x_batch[2,2:end, :] .- x_batch[2,1:end-1, :]) ./ (x_batch[1,2:end, :] .- x_batch[1,1:end-1, :])
        U_dot_nominal_traj[1, end, :] = (x_batch[2,end, :] .- x_batch[2,end-1, :]) ./ (x_batch[1, end, :] .- x_batch[1, end-1, :])
    end

    T_traj = x_batch[1:1,:,:]
    U_dot_safe_traj = copy(U_dot_nominal_traj)

    for i in eachindex(x_batch[1,2:end,1]) # 1,2,3,4...50
        if i + max_mpc_step  > size(T_traj,2)
            mpc_end = size(T_traj,2)
        else
            mpc_end = i + max_mpc_step
        end
        if use_central_flag
            U_pred_traj = reconstruct_traj_central(U_0, T_traj, U_dot_safe_traj)
        else
            U_pred_traj = reconstruct_traj(U_0, T_traj, U_dot_safe_traj)
        end

        Ut_pred_traj = cat(T_traj, U_pred_traj, dims=1)
        Yt_pred_traj, ∇ϕ = Zygote.pullback(model_NO, Ut_pred_traj)
        G_u = ∇ϕ(ones(size(T_traj)))[1][2, i:mpc_end,1]
        G_t = ∇ϕ(ones(size(T_traj)))[1][1, i:mpc_end,1]

        Yt = cat(T_traj, Yt_pred_traj, dims=1) # NO
        Yt = reshape(Yt, (size(Yt)[1], size(Yt)[2]*size(Yt)[3]))
        state_dim, batchsize = size(Yt) # 2*51000
        _, ∇ϕ = Zygote.pullback(model_CBF, Yt)
        ∇ϕ_Y = ∇ϕ(ones(size(Yt)))[1] ./ state_dim

        phi_t = ∇ϕ_Y[1, i:mpc_end]
        phi_Y = ∇ϕ_Y[2, i:mpc_end]
        
        phiY = Yt_pred_traj[1,i:mpc_end,1]

        phiU_0 = model_CBF([t_0,U_0])[1]

        U_dot_safe = solve_multistep_qp_with_cplex(U_dot_nominal_traj[1,i:mpc_end,1],G_u,G_t,phi_t,phi_Y,phiY,α,T,phiU_0;opt_alpha=opt_alpha,diff_U_dot_threshold=diff_U_dot_threshold)

        U_dot_safe_traj[i] = U_dot_safe
        
    end
    if use_central_flag
        push!(filter_result_list_ppo, (reconstruct_traj_central(U_0, T_traj, U_dot_safe_traj), x_batch, y_batch, safe_batch))
    else
        push!(filter_result_list_ppo, (reconstruct_traj(U_0, T_traj, U_dot_safe_traj), x_batch, y_batch, safe_batch))
    end
    ind += 1
end




In [ ]:
using NPZ
U_safe = [filter_result_list_ppo[i][1][1,:,1] for i in 1:100]
U_safe = reduce(hcat, U_safe)
@show U_safe[:,1]
U_nominal = [filter_result_list_ppo[i][2][2,:,1] for i in 1:100]
U_nominal = reduce(hcat, U_nominal)
@show U_nominal[:,1]
Y_nominal = [filter_result_list_ppo[i][3][1,:,1] for i in 1:100]
Y_nominal = reduce(hcat, Y_nominal)
@show Y_nominal[:,1]
safe_label = [filter_result_list_ppo[i][4][:,1] for i in 1:100]
safe_label = reduce(hcat, safe_label)
@show safe_label[:,1]

npzwrite("ns_ppo_filter_result.npy", Dict(
	"U_safe" => U_safe,
	"U_nominal" => U_nominal,
    "Y_nominal" => Y_nominal,
    "safe_label" => safe_label
))
